In [7]:
from EvGym.charge_world import ChargeWorldEnv, Session
import pandas as pd
import time
from tqdm.notebook import tqdm
from IPython.display import clear_output

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', None)

In [8]:
df_data = pd.read_csv("data/prepared_elaad_transactions.csv", parse_dates=["TransactionStartDT", "TransactionStopDT"])
df_data["TransactionStartDate"] = pd.to_datetime(df_data["TransactionStartDate"]).dt.date
df_data["TransactionStopDate"] = pd.to_datetime(df_data["TransactionStopDate"]).dt.date
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   TransactionId            10000 non-null  int64         
 1   ChargePoint              10000 non-null  object        
 2   Connector                10000 non-null  int64         
 3   TransactionStartDT       10000 non-null  datetime64[ns]
 4   TransactionStopDT        10000 non-null  datetime64[ns]
 5   StartCard                10000 non-null  object        
 6   ConnectedTime            10000 non-null  float64       
 7   ChargeTime               10000 non-null  float64       
 8   TotalEnergy              10000 non-null  float64       
 9   MaxPower                 10000 non-null  float64       
 10  BatteryCapacity          10000 non-null  int64         
 11  SOC_arr                  10000 non-null  float64       
 12  TransactionStartDate     10000 no

In [9]:
df_data[df_data["TransactionStopTS"] == 348].sort_values(["TransactionStartTS"]).head()

,TransactionId,ChargePoint,Connector,TransactionStartDT,TransactionStopDT,StartCard,ConnectedTime,ChargeTime,TotalEnergy,MaxPower,...,TransactionStopSecDay,TransactionStartHourDay,TransactionStopHourDay,TransactionStartAbsSec,TransactionStopAbsSec,TransactionStartTS,TransactionStopTS,ConnectedTimeMin,measConnectedTimeMin,errConnectedTime


In [10]:
world = ChargeWorldEnv(df_data)
world.df_active.head(200)

,idSess,C,t_arr,soc_arr,E_arr,t_dep,E_req,t_soj,E,soc,t_rem
0,-1,0,0,0,0,0,0,0,0,0,0
1,-1,0,0,0,0,0,0,0,0,0,0
2,-1,0,0,0,0,0,0,0,0,0,0
3,-1,0,0,0,0,0,0,0,0,0,0
4,-1,0,0,0,0,0,0,0,0,0,0
5,-1,0,0,0,0,0,0,0,0,0,0
6,-1,0,0,0,0,0,0,0,0,0,0
7,-1,0,0,0,0,0,0,0,0,0,0
8,-1,0,0,0,0,0,0,0,0,0,0
9,-1,0,0,0,0,0,0,0,0,0,0


In [11]:
for i in tqdm(range(df_data["TransactionStopTS"].max().astype(int))):
    world.step()

  0%|          | 0/35103 [00:00<?, ?it/s]

In [12]:
world.step()
print(world.t)
world.df_active.head(200)

35104


,idSess,C,t_arr,soc_arr,E_arr,t_dep,E_req,t_soj,E,soc,t_rem
0,-1,0,0,0,0,0,0,0,0,0,0
1,-1,0,0,0,0,0,0,0,0,0,0
2,-1,0,0,0,0,0,0,0,0,0,0
3,-1,0,0,0,0,0,0,0,0,0,0
4,-1,0,0,0,0,0,0,0,0,0,0
5,-1,0,0,0,0,0,0,0,0,0,0
6,-1,0,0,0,0,0,0,0,0,0,0
7,-1,0,0,0,0,0,0,0,0,0,0
8,-1,0,0,0,0,0,0,0,0,0,0
9,-1,0,0,0,0,0,0,0,0,0,0


In [13]:
world.df_occupied.head()

,idSess,C,t_arr,soc_arr,E_arr,t_dep,E_req,t_soj,E,soc,t_rem
0,3634012,80,35012,0.09875,7.9,35103,72.1,91,7.9,0.09875,91


In [14]:
world.df_occupied[world.df_occupied["idSess"] == 3261657].index[0]


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
t = 0
df_arrivals = df_data[df_data["TransactionStartTS"] == t]
for i, arr_car in df_arrivals.iterrows():
    print(i, arr_car.TransactionId)

9458 3261657


In [ ]:
# Car is now a list
columns=["Name", "age", "height"]
df_people = pd.DataFrame(columns=columns)

In [ ]:
l_people = []
for i in range(10):
    l_people += [[f"Josh{i}", i*6, 190-2*i]]

df_ind = pd.concat((df_people, pd.DataFrame(l_people, columns=columns)))



In [ ]:
df_ind.head(200)

,Name,age,height
0,Josh0,0,190
1,Josh1,6,188
2,Josh2,12,186
3,Josh3,18,184
4,Josh4,24,182
5,Josh5,30,180
6,Josh6,36,178
7,Josh7,42,176
8,Josh8,48,174
9,Josh9,54,172


In [ ]:
df_older = df_ind[df_ind["age"] > 43]
df_oldest = df_older[df_older["age"] == 48]

In [ ]:
idx = df_ind[df_ind["age"] > 80].index[0]
df_ind.loc[idx, ["Name", "age", "height"]] = ["Henry", 25, 179]


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
df_ind.drop(index=(df_ind[df_ind["Name"] == "Josh1"]).index).head(1000).reset_index(drop=True)

,Name,age,height
0,Josh0,0,190
1,Josh2,12,186
2,Josh3,18,184
3,Josh4,24,182
4,Josh5,30,180
5,Josh6,36,178
6,Henry,25,179
7,Henry,25,179
8,Henry,25,179


In [ ]:
# Example of custom environment

#class ShowerEnv(Env):
#    def __init__(self):
#        # Actions we can take, down, stay, up
#        self.action_space = Discrete(3)
#        # Temperature array
#        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
#        # Set start temp
#        self.state = 38 + random.randint(-3,3)
#        # Set shower length
#        self.shower_length = 60
#        
#    def step(self, action):
#        # Apply action
#        # 0 -1 = -1 temperature
#        # 1 -1 = 0 
#        # 2 -1 = 1 temperature 
#        self.state += action -1 
#        # Reduce shower length by 1 second
#        self.shower_length -= 1 
#        
#        # Calculate reward
#        if self.state >=37 and self.state <=39: 
#            reward =1 
#        else: 
#            reward = -1 
#        
#        # Check if shower is done
#        if self.shower_length <= 0: 
#            done = True
#        else:
#            done = False
#        
#        # Apply temperature noise
#        #self.state += random.randint(-1,1)
#        # Set placeholder for info
#        info = {}
#        
#        # Return step information
#        return self.state, reward, done, info
#
#    def render(self):
#        # Implement viz
#        pass
#    
#    def reset(self):
#        # Reset shower temperature
#        self.state = 38 + random.randint(-3,3)
#        # Reset shower time
#        self.shower_length = 60 
#        return self.state
#    
